In [2]:
import pyspark.pandas as ps

path = "abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/bronze/Nationwide/"

StatementMeta(sparkctprdnc, 0, 2, Finished, Available)

In [3]:
def deep_ls(path: str, max_depth=4):
    """
    List all files and folders in specified path and
    subfolders within maximum recursion depth.
    """

    # List all files in path
    li = mssparkutils.fs.ls(path)

    # Return all files
    for x in li:
        if x.size != 0:
            yield x

    # If the max_depth has not been reached, start
    # listing files and folders in subdirectories
    if max_depth > 1:
        for x in li:
            if x.size != 0:
                continue
            for y in deep_ls(x.path, max_depth - 1):
                yield y

    # If max_depth has been reached,
    # return the folders
    else:
        for x in li:
            if x.size == 0:
                yield x
 # If the max_depth has not been reached, start
    # listing files and folders in subdirectories
    if max_depth > 1:
        for x in li:
            if x.size != 0:
                continue
            for y in deep_ls(x.path, max_depth - 1):
                yield y

    # If max_depth has been reached,
    # return the folders
    else:
        for x in li:
            if x.size == 0:
                yield x


StatementMeta(sparkctprdnc, 0, 3, Finished, Available)

In [4]:
files = []
files_generator = deep_ls(path)
for file in files_generator:
    files.append(file)

StatementMeta(sparkctprdnc, 0, 4, Finished, Available)

In [5]:
dataframe_list = []
for file in files:
    path_name = int(file.name.replace(".xlsx",""))
    spdf = ps.read_excel(file.path, skiprows =1)
    spdf["date_int"] = path_name
    dataframe_list.append(spdf)


StatementMeta(sparkctprdnc, 0, 5, Finished, Available)

/home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [22]:
spdfNationwideSilver = ps.concat(dataframe_list)
spdfNationwideSilver.reset_index(inplace = True)

StatementMeta(sparkctprdnc, 0, 22, Finished, Available)

In [30]:
new_names = ['idx', 'AgencyNumber', 'NotificationType', 'DateVal', 'Insured',
       'BillingAccountNumber', 'Policy', 'Status', 'CancelDate', 'Company',
       'PaymentAcceptanceDate', 'TotalBalance', 'MinimumDue',
       'NonSufficientFunds', 'DateReinstated', 'CancellationMailed',
       'dateint']
spdfNationwideSilver.columns = new_names

StatementMeta(sparkctprdnc, 0, 30, Finished, Available)

In [31]:
sdfNationwideSilver = spdfNationwideSilver.to_spark()

StatementMeta(sparkctprdnc, 0, 31, Finished, Available)

In [32]:
path = "abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/silver/nationwide"
sdfNationwideSilver.write.format("delta").option("overwrite","true").save(path)

StatementMeta(sparkctprdnc, 0, 32, Finished, Available)